# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [59]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from catboost import CatBoostRegressor
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

In [60]:
df=pd.read_csv('/datasets/autos.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [61]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [62]:
print(df['DateCreated'].min())
print(df['DateCreated'].max())

2014-03-10 00:00:00
2016-04-07 00:00:00


Можно сразу удалить ненужные столбцы. На цену машины точно не влияют:  
- DataCrawled
- RegistrationMonth (год важен, месяц скорее всего лишняя информация)
- Brand (у нас есть модель машины, нам не нужен ее бренд, т.к. названия моделей уникальные)
- DateCreated (косвенно на цену влияет разница между датой создания и датой регистрации, в датасете период в 2 года, скорее всего это не повлияет на результат)
- NumberOfPictures
- PostalCode
- LastSeen

In [63]:
df.duplicated().value_counts()#есть полные дубликаты, их надо удалить сейчас. Остальные дубликаты скорее всего просто совпадения

False    354365
True          4
dtype: int64

In [64]:
df=df.drop_duplicates()

In [65]:
#смотрю на самые часто повторяющиеся данные
for i in df.columns:
    print(df[i].value_counts())

2016-03-24 14:49:47    7
2016-03-26 22:57:31    6
2016-03-19 21:49:56    6
2016-03-09 16:48:39    5
2016-03-22 10:42:10    5
                      ..
2016-03-14 21:55:05    1
2016-03-15 13:56:59    1
2016-03-31 09:51:00    1
2016-03-16 11:45:47    1
2016-03-12 16:55:36    1
Name: DateCrawled, Length: 271174, dtype: int64
0        10772
500       5670
1500      5394
1000      4648
1200      4594
         ...  
13180        1
10879        1
2683         1
634          1
8188         1
Name: Price, Length: 3731, dtype: int64
sedan          91457
small          79830
wagon          65165
bus            28775
convertible    20203
coupe          16161
suv            11996
other           3288
Name: VehicleType, dtype: int64
2000    24490
1999    22727
2005    22109
2001    20123
2006    19900
        ...  
4100        1
1200        1
5300        1
8888        1
2290        1
Name: RegistrationYear, Length: 151, dtype: int64
manual    268249
auto       66283
Name: Gearbox, dtype: int64
0     

Пройдемся по странностям:
1. 0 цена, по сути NaN для наших целей. Узнать не сможем, придется удалить строки
2. "other" в некоторых столбцах. Во всех случаях может значительно влиять на цену, придется удалить.
3. Странный год регистрации. Судя по всему мы находимся в 2016, все после этого стоит удалить.
4. 0 мощность, тоже может значительно влиять на цену (разные конфигурации модели), придется удалить.
5. Высокие мощности. Даже в премиум моделях редко будет выше 1000, придется удалить.

In [66]:
df1=df.drop(['DateCrawled','RegistrationMonth','Brand','NumberOfPictures','PostalCode','LastSeen', 'DateCreated'], axis=1)
df2=df1[(df1['Price'] >= 100) & (df1['Power'] != 0)]#убираю нули, отсчечка цены в 100 евро
#решил поставить отсечку на 1500 мощности и 1960 году
df2=df2[(df2['RegistrationYear'] <= 2016) & (df2['RegistrationYear'] >= 1960) & (df2['Power'] <= 1500)]
for i in df2.columns:#цикл убирает все other
    df2=df2[df2[i] != 'other']
df2.reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273075 entries, 1 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             273075 non-null  int64 
 1   VehicleType       262725 non-null  object
 2   RegistrationYear  273075 non-null  int64 
 3   Gearbox           268166 non-null  object
 4   Power             273075 non-null  int64 
 5   Model             262303 non-null  object
 6   Kilometer         273075 non-null  int64 
 7   FuelType          258639 non-null  object
 8   Repaired          233547 non-null  object
dtypes: int64(4), object(5)
memory usage: 20.8+ MB


Уже до работы с пропусками пришлось удалить 22% строк. Неприятно, но не вижу вариантов заполнения этих данных чем-то еще. Т.к. сейчас в датафрейме остались только нужные для модели признаки, пропуски тоже придется удалить.

In [67]:
df2['Repaired']=df2['Repaired'].fillna('no')

In [68]:
df2=df2.dropna(subset=['Model']).reset_index(drop=True)

In [69]:
#создаю словарь самых часто встречающихся кузовов для каждой модели
unique_models=df2['Model'].unique()
common_type={}
for i in unique_models:
    max_type=df2[df2['Model']==i]['VehicleType'].value_counts().index[0]
    common_type[i]=max_type

In [70]:
#цикл дает всем nan значения на основе словаря
#достаточно медленный цикл, скорее всего есть более элегатное решение
for i in range(len(df2['VehicleType'])):
    if pd.isna(df2.iloc[i,1])==True:
        df2.iloc[i,1]=common_type[df2.iloc[i,5]]

In [71]:
df_final=df2.dropna().reset_index(drop=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247035 entries, 0 to 247034
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             247035 non-null  int64 
 1   VehicleType       247035 non-null  object
 2   RegistrationYear  247035 non-null  int64 
 3   Gearbox           247035 non-null  object
 4   Power             247035 non-null  int64 
 5   Model             247035 non-null  object
 6   Kilometer         247035 non-null  int64 
 7   FuelType          247035 non-null  object
 8   Repaired          247035 non-null  object
dtypes: int64(4), object(5)
memory usage: 17.0+ MB


Потеряли 39% строк из изначального датафрейма. С одной стороны это плохо, однако это скорее улучшит качество предсказаний, и точно увеличит скорость обучения модели, что важно для клиента. Используем OrdinalEncoding для преобразования признаков.

In [72]:
df['Repaired'].value_counts()

no     247158
yes     36053
Name: Repaired, dtype: int64

In [73]:
df[df['Power']==0].shape

(40225, 16)

## Обучение моделей

In [74]:
df_train, df_valid, df_test = np.split(df_final.sample(frac=1, random_state=123), 
                                       [int(.6*len(df_final)), int(.8*len(df_final))])
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(148221, 9)
(49407, 9)
(49407, 9)


In [75]:
features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']

Закодируем данные.

In [76]:
encoder_oe=OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
features_train = pd.DataFrame(encoder_oe.fit_transform(features_train),
                            columns=features_train.columns)
features_valid=pd.DataFrame(encoder_oe.transform(features_valid),
                            columns=features_valid.columns)
features_test=pd.DataFrame(encoder_oe.transform(features_test),
                            columns=features_test.columns)

Сначала обучим RandomForestRegressor.

RMSE выше 2500, посмотрим как поведут себя модели с бустингом.

In [77]:
train_data = lgb.Dataset(features_train, target_train)
validation_data = lgb.Dataset(features_valid, target_valid, reference=train_data)

In [83]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate':0.1,
    'num_leaves': 31,
}

In [85]:
%%time
gbm = lgb.train(params, train_data, num_boost_round=50, valid_sets=validation_data)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.300751 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 581
[LightGBM] [Info] Number of data points in the train set: 148221, number of used features: 8
[LightGBM] [Info] Start training from score 5063.517140
[1]	valid_0's rmse: 4320.02
[2]	valid_0's rmse: 4014.87
[3]	valid_0's rmse: 3746.01
[4]	valid_0's rmse: 3511.65
[5]	valid_0's rmse: 3307.16
[6]	valid_0's rmse: 3124.66
[7]	valid_0's rmse: 2965.01
[8]	valid_0's rmse: 2825.38
[9]	valid_0's rmse: 2704.77
[10]	valid_0's rmse: 2597.48
[11]	valid_0's rmse: 2507.41
[12]	valid_0's rmse: 2426.9
[13]	valid_0's rmse: 2358.12
[14]	valid_0's rmse: 2297.69
[15]	valid_0's rmse: 2246.39
[16]	valid_0's rmse: 2200.55
[17]	valid_0's rmse: 2160.03
[18]	valid_0's rmse: 2124.2
[19]	valid_0's rmse: 2092.14
[20]	valid_0's rmse: 2064.2
[21]	valid_0's rmse: 

In [90]:
model = CatBoostRegressor(iterations=1000,
                           eval_metric='RMSE',
                           max_leaves = 31,
                           learning_rate = 0.1,
                           verbose=100)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
mean_squared_error(target_valid, predictions_valid, squared=False)

0:	learn: 4339.6272215	total: 19.2ms	remaining: 19.1s
100:	learn: 1804.0039089	total: 2.03s	remaining: 18s
200:	learn: 1688.1642594	total: 4.11s	remaining: 16.4s
300:	learn: 1626.7719657	total: 6.25s	remaining: 14.5s
400:	learn: 1585.1412928	total: 8.45s	remaining: 12.6s
500:	learn: 1554.4463731	total: 10.5s	remaining: 10.5s
600:	learn: 1531.9062416	total: 12.8s	remaining: 8.5s
700:	learn: 1512.5913365	total: 14.9s	remaining: 6.34s
800:	learn: 1496.1819727	total: 16.8s	remaining: 4.18s
900:	learn: 1480.9754131	total: 18.9s	remaining: 2.08s
999:	learn: 1468.2504331	total: 20.9s	remaining: 0us


1541.2032505562684

## Анализ моделей

Еще раз пройдемся по всем полученным моделям и сравним их по каждому параметру.

In [100]:
%%time
model=RandomForestRegressor(random_state=12345, n_estimators=4, max_depth=7)
model.fit(features_train, target_train)

CPU times: user 490 ms, sys: 4.02 ms, total: 494 ms
Wall time: 493 ms


RandomForestRegressor(max_depth=7, n_estimators=4, random_state=12345)

In [101]:
%%time
predictions_valid = model.predict(features_valid)

CPU times: user 20.9 ms, sys: 9 µs, total: 20.9 ms
Wall time: 18.4 ms


In [102]:
mean_squared_error(target_valid, predictions_valid, squared=False)

2080.860271899557

In [103]:
%%time
gbm = lgb.train(params, train_data, num_boost_round=50, valid_sets=validation_data)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.315101 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 581
[LightGBM] [Info] Number of data points in the train set: 148221, number of used features: 8
[LightGBM] [Info] Start training from score 5063.517140
[1]	valid_0's rmse: 4320.02
[2]	valid_0's rmse: 4014.87
[3]	valid_0's rmse: 3746.01
[4]	valid_0's rmse: 3511.65
[5]	valid_0's rmse: 3307.16
[6]	valid_0's rmse: 3124.66
[7]	valid_0's rmse: 2965.01
[8]	valid_0's rmse: 2825.38
[9]	valid_0's rmse: 2704.77
[10]	valid_0's rmse: 2597.48
[11]	valid_0's rmse: 2507.41
[12]	valid_0's rmse: 2426.9
[13]	valid_0's rmse: 2358.12
[14]	valid_0's rmse: 2297.69
[15]	valid_0's rmse: 2246.39
[16]	valid_0's rmse: 2200.55
[17]	valid_0's rmse: 2160.03
[18]	valid_0's rmse: 2124.2
[19]	valid_0's rmse: 2092.14
[20]	valid_0's rmse: 2064.2
[21]	valid_0's rmse: 

In [104]:
%%time
predictions_valid = gbm.predict(features_valid, num_iteration=gbm.best_iteration)

CPU times: user 220 ms, sys: 3.76 ms, total: 223 ms
Wall time: 270 ms


In [105]:
mean_squared_error(target_valid, predictions_valid, squared=False)

1758.101593546505

In [107]:
%%time
model = CatBoostRegressor(iterations=1000,
                           eval_metric='RMSE',
                           max_leaves = 31,
                           learning_rate = 0.1,
                           verbose=100)
model.fit(features_train, target_train)

0:	learn: 4339.6272215	total: 21ms	remaining: 21s
100:	learn: 1804.0039089	total: 2.11s	remaining: 18.8s
200:	learn: 1688.1642594	total: 4.19s	remaining: 16.7s
300:	learn: 1626.7719657	total: 6.26s	remaining: 14.5s
400:	learn: 1585.1412928	total: 8.35s	remaining: 12.5s
500:	learn: 1554.4463731	total: 10.2s	remaining: 10.2s
600:	learn: 1531.9062416	total: 12.2s	remaining: 8.07s
700:	learn: 1512.5913365	total: 14.1s	remaining: 6.02s
800:	learn: 1496.1819727	total: 16s	remaining: 3.99s
900:	learn: 1480.9754131	total: 18s	remaining: 1.98s
999:	learn: 1468.2504331	total: 20s	remaining: 0us
CPU times: user 19.9 s, sys: 299 ms, total: 20.2 s
Wall time: 21.1 s


In [108]:
%%time
predictions_valid = model.predict(features_valid)

CPU times: user 55.8 ms, sys: 0 ns, total: 55.8 ms
Wall time: 54.2 ms


In [109]:
mean_squared_error(target_valid, predictions_valid, squared=False)

1541.2032505562684

Т.к. клиент не обозначил приоритетность критериев, предположим, что они все равны. Оценим все модели от 1 до 3 (т.к. модели 3) по каждому критерию, самое большое кол-во баллов определит победителя. Критерии по порядку - качество предсказания, время обучения, время предсказания. Оценки:
* sklearn.RandomForestRegressor - 1, 3, 3
* LightGBM - 2, 1, 1
* CatBoost - 3, 2, 2

RandomForestRegressor и CatBoost обладают одинаковым кол-вом баллов. Финальный выбор зависит от клиента и его запросов. Скорее всего модель не следует переобучать каждый раз, когда пользователю нужно показать примерную стоимость его машины, так что скорость обучения CatBoost в 21 секунду достаточна. Скорость предсказаний у обоих меньше 100ms, человек не заметит разницы в скорости. При этом CatBoost на 33% точнее, что скорее всего и будет являться самым важным параметром. Протестируем модель.

In [110]:
%%time
model = CatBoostRegressor(iterations=1000,
                           eval_metric='RMSE',
                           max_leaves = 31,
                           learning_rate = 0.1,
                           verbose=100)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
mean_squared_error(target_test, predictions_test, squared=False)

0:	learn: 4339.6272215	total: 24.3ms	remaining: 24.3s
100:	learn: 1804.0039089	total: 2.11s	remaining: 18.8s
200:	learn: 1688.1642594	total: 4.25s	remaining: 16.9s
300:	learn: 1626.7719657	total: 6.31s	remaining: 14.7s
400:	learn: 1585.1412928	total: 8.51s	remaining: 12.7s
500:	learn: 1554.4463731	total: 10.6s	remaining: 10.6s
600:	learn: 1531.9062416	total: 12.7s	remaining: 8.42s
700:	learn: 1512.5913365	total: 14.9s	remaining: 6.34s
800:	learn: 1496.1819727	total: 17s	remaining: 4.21s
900:	learn: 1480.9754131	total: 19.1s	remaining: 2.1s
999:	learn: 1468.2504331	total: 21.1s	remaining: 0us
CPU times: user 21.2 s, sys: 228 ms, total: 21.4 s
Wall time: 22.8 s


1534.9147365267916

Модель хорошо справилась с тестовой выборкой, остановимся на ней.